In [1]:
from db_tools.setup import setup
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql import functions as F

In [2]:
spark = setup()

23/10/29 18:21:15 WARN Utils: Your hostname, luan-Dell-G15-5520 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/10/29 18:21:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/29 18:21:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/29 18:21:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/29 18:21:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/10/29 18:21:16 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/10/29 18:21:16 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


SparkSession available as "spark"


In [3]:
df = spark.sql("""SELECT * FROM film""")

In [7]:
df.select('title',
          'length',
          F.when(
              (F.col('length')>0)
              & (F.col('length')<=50),'Short'
          )\
          .when((F.col('length')>50) 
              & (F.col('length')<=120),'Medium'
          ).otherwise('Long').alias('duration')
         ).orderBy('title').show()

+-------------------+------+--------+
|              title|length|duration|
+-------------------+------+--------+
|   Academy Dinosaur|    86|  Medium|
|     Ace Goldfinger|    48|   Short|
|   Adaptation Holes|    50|   Short|
|   Affair Prejudice|   117|  Medium|
|        African Egg|   130|    Long|
|       Agent Truman|   169|    Long|
|    Airplane Sierra|    62|  Medium|
|    Airport Pollock|    54|  Medium|
|      Alabama Devil|   114|  Medium|
|   Aladdin Calendar|    63|  Medium|
|    Alamo Videotape|   126|    Long|
|     Alaska Phantom|   136|    Long|
|        Ali Forever|   150|    Long|
|     Alice Fantasia|    94|  Medium|
|       Alien Center|    46|   Short|
|    Alley Evolution|   180|    Long|
|         Alone Trip|    82|  Medium|
|      Alter Victory|    57|  Medium|
|       Amadeus Holy|   113|  Medium|
|Amelie Hellfighters|    79|  Medium|
+-------------------+------+--------+
only showing top 20 rows



# CASE WHEN with aggregates

In [9]:
df.select(F.sum(F.when(F.col('rental_rate') == 0.99,1).otherwise(0)).alias('Economy'),
          F.sum(F.when(F.col('rental_rate') == 2.99,1).otherwise(0)).alias('Mass'),
          F.sum(F.when(F.col('rental_rate') == 4.99,1).otherwise(0)).alias('Premium')).show()
          

+-------+----+-------+
|Economy|Mass|Premium|
+-------+----+-------+
|    341| 323|    336|
+-------+----+-------+



In [13]:
df.select('title',
          'rating',
          F.when(F.col('rating') == 'G','General Audiences')\
          .when(F.col('rating') == 'PG','Parental Guidance Suggested')\
          .when(F.col('rating') == 'PG-13','Parents Strongly Cautioned')\
          .when(F.col('rating') == 'R', 'Restricted')\
          .when(F.col('rating') == 'NC-17', 'Adults Only').alias('rating_description')
         ).orderBy('title').show()

+-------------------+------+--------------------+
|              title|rating|  rating_description|
+-------------------+------+--------------------+
|   Academy Dinosaur|    PG|Parental Guidance...|
|     Ace Goldfinger|     G|   General Audiences|
|   Adaptation Holes| NC-17|         Adults Only|
|   Affair Prejudice|     G|   General Audiences|
|        African Egg|     G|   General Audiences|
|       Agent Truman|    PG|Parental Guidance...|
|    Airplane Sierra| PG-13|Parents Strongly ...|
|    Airport Pollock|     R|          Restricted|
|      Alabama Devil| PG-13|Parents Strongly ...|
|   Aladdin Calendar| NC-17|         Adults Only|
|    Alamo Videotape|     G|   General Audiences|
|     Alaska Phantom|    PG|Parental Guidance...|
|        Ali Forever|    PG|Parental Guidance...|
|     Alice Fantasia| NC-17|         Adults Only|
|       Alien Center| NC-17|         Adults Only|
|    Alley Evolution| NC-17|         Adults Only|
|         Alone Trip|     R|          Restricted|


In [14]:
df.select(F.sum(F.when(F.col('rating') == 'G',1).otherwise(0)).alias('General_Audiences'),
          F.sum(F.when(F.col('rating') == 'PG',1).otherwise(0)).alias('Parental_Guidance_Suggested'),
          F.sum(F.when(F.col('rating') == 'PG-13',1).otherwise(0)).alias('Parents_Strongly_Cautioned'),
          F.sum(F.when(F.col('rating') == 'R',1).otherwise(0)).alias('Restricted'),
          F.sum(F.when(F.col('rating') == 'NC-17',1).otherwise(0)).alias('Adults Only')).show()

+-----------------+---------------------------+--------------------------+----------+-----------+
|General_Audiences|Parental_Guidance_Suggested|Parents_Strongly_Cautioned|Restricted|Adults Only|
+-----------------+---------------------------+--------------------------+----------+-----------+
|              178|                        194|                       223|       195|        210|
+-----------------+---------------------------+--------------------------+----------+-----------+



# COALESCE: not work as you expect that does

## From pyspark API

In [32]:
cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
cDf.select(F.coalesce(cDf["a"], cDf["b"])).show()

+--------------+
|coalesce(a, b)|
+--------------+
|          null|
|             1|
|             2|
+--------------+



# IFNULL (Pyspark 3.5.0 and on exclusive)

In [36]:
df = spark.createDataFrame([(None,), (1,)], ["e"])
df.select(F.ifnull(df.e, sf.lit(8))).show()

AttributeError: module 'pyspark.sql.functions' has no attribute 'ifnull'

# NULLIF (Pyspark 3.5.0 and on exclusive)

In [35]:
df = spark.createDataFrame([(None, None,), (1, 9,)], ["a", "b"])
df.select(F.nullif(df.a, df.b).alias('r')).collect()

AttributeError: module 'pyspark.sql.functions' has no attribute 'nullif'

# CAST

In [37]:
df = spark.sql("""SELECT * FROM rental""")
df.select('rental_id',
          F.col('rental_date').cast('date').alias('rental_date_as_date'),
          F.col('rental_date').cast('timestamp').alias('rental_date_as_timestamp')).show()

+---------+-------------------+------------------------+
|rental_id|rental_date_as_date|rental_date_as_timestamp|
+---------+-------------------+------------------------+
|        2|         2005-05-24|     2005-05-24 22:54:33|
|        3|         2005-05-24|     2005-05-24 23:03:39|
|        4|         2005-05-24|     2005-05-24 23:04:41|
|        5|         2005-05-24|     2005-05-24 23:05:21|
|        6|         2005-05-24|     2005-05-24 23:08:07|
|        7|         2005-05-24|     2005-05-24 23:11:53|
|        8|         2005-05-24|     2005-05-24 23:31:46|
|        9|         2005-05-25|     2005-05-25 00:00:40|
|       10|         2005-05-25|     2005-05-25 00:02:21|
|       11|         2005-05-25|     2005-05-25 00:09:02|
|       12|         2005-05-25|     2005-05-25 00:19:27|
|       13|         2005-05-25|     2005-05-25 00:22:55|
|       14|         2005-05-25|     2005-05-25 00:31:15|
|       15|         2005-05-25|     2005-05-25 00:39:22|
|       16|         2005-05-25|